[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pugapatricia/gestion-documentaria-para-pymes/blob/main/etiquetado/Generador_etiquetasHF.ipynb)

[![Ver en GitHub](https://img.shields.io/badge/GitHub-Repo-black?logo=github)](https://github.com/pugapatricia/gestion-documentaria-para-pymes/tree/main/etiquetado)

#Importaciones

In [1]:
!pip install -q requests msal python-docx PyPDF2 pandas openpyxl python-pptx openai

In [2]:
import requests
import msal
from docx import Document
import PyPDF2
import pandas as pd
from pptx import Presentation
import os
import spacy
import torch
from openai import OpenAI
import re
from transformers import pipeline

# Configuración

In [3]:
CLIENT_ID = "e3f2393e-7348-47d1-9c64-8d8efe6a5e95"
AUTHORITY = "https://login.microsoftonline.com/consumers"
SCOPE = ["User.Read", "Files.ReadWrite"]

ext_permitidas = {"pdf", "docx", "xlsx", "xls", "pptx", "txt", "csv"}
url = "https://graph.microsoft.com/v1.0/me/drive/root:/Etiquetados:/children"

etiquetas_global = set()
# device=0 usa la primera GPU, device=-1 fuerza CPU
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


# Conección con OneDrive

In [4]:
app = msal.PublicClientApplication(CLIENT_ID, authority=AUTHORITY)

flow = app.initiate_device_flow(scopes=SCOPE)
if "user_code" not in flow:
    raise Exception("No se pudo iniciar el device flow. Revisa tu configuración en Azure.")

print(flow["message"])  # 👉 Copia el código en https://microsoft.com/devicelogin
result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception(f"Error autenticación: {result.get('error_description')}")

access_token = result["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}

# Llamada a la API con tu token de acceso
resp = requests.get(url, headers=headers)
if resp.status_code != 200:
    raise Exception(f"Error al obtener archivos: {resp.text}")
data = resp.json()


To sign in, use a web browser to open the page https://www.microsoft.com/link and enter the code PREP256R to authenticate.


# Funciones

Procesamos cada documento de acuerdo con su tipo (PDF, Word, Excel, etc.) para convertirlo en datos estructurados. Este proceso se realiza mediante las siguientes funciones.

In [5]:
def leer_pdf(file_path):
    text = ""
    with open(file_path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text

def leer_docx(file_path):
    doc = Document(file_path)
    return "\n".join([p.text for p in doc.paragraphs])

def leer_excel(file_path):
    try:
        df_dict = pd.read_excel(file_path, sheet_name=None)
    except Exception as e:
        return f"Error leyendo Excel: {e}"

    texto = []
    for nombre, hoja in df_dict.items():
        hoja = hoja.fillna("")  # reemplaza NaN por ""
        texto.append(f"\n--- Hoja: {nombre} ---\n")
        texto.append(hoja.to_string())
    return "\n".join(texto)

def leer_csv(file_path):
    df = pd.read_csv(file_path)
    return df.to_string()

def leer_pptx(file_path):
    prs = Presentation(file_path)
    texto = []
    for i, slide in enumerate(prs.slides, 1):
        texto.append(f"\n--- Diapositiva {i} ---\n")
        for shape in slide.shapes:
            if hasattr(shape, "text_frame") and shape.text_frame:
                texto.append(shape.text_frame.text)
    return "\n".join(texto)

def leer_txt(file_path):
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        return f.read()

def procesar_archivo_temporal(item):
    nombre = item.get("name")
    ext = nombre.split(".")[-1].lower()
    if ext not in ext_permitidas:
        return ""

Función para solicitar a OpenAI la generación de etiquetas/tickers por documento, con un límite máximo

In [6]:

def sugerir_tickers(texto, max_etiquetas=10):
    if not texto.strip():
        return []

    candidate_labels = texto.split()
    resultado = classifier(texto, candidate_labels, multi_label=True)

    etiquetas_lista = [word.lower() for word, score in zip(resultado['labels'], resultado['scores'])][:max_etiquetas]
    etiquetas_global.update(etiquetas_lista)
    return etiquetas_lista


Descarga un archivo temporal desde una URL, verifica que su extensión esté permitida y lo lee según su tipo, devolviendo su contenido como texto.

In [7]:
def procesar_archivo_temporal(item):
    nombre = item.get("name")
    ext = nombre.split(".")[-1].lower()
    if ext not in ext_permitidas:
        return ""

    tmp_path = f"/tmp/{nombre}"
    try:
        r = requests.get(item["@microsoft.graph.downloadUrl"], stream=True)
        r.raise_for_status()
        with open(tmp_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

        if ext == "pdf":
            return leer_pdf(tmp_path)
        elif ext == "docx":
            return leer_docx(tmp_path)
        elif ext in {"xlsx", "xls"}:
            return leer_excel(tmp_path)
        elif ext == "pptx":
            return leer_pptx(tmp_path)
        elif ext == "txt":
            return leer_txt(tmp_path)
        elif ext == "csv":
            return leer_csv(tmp_path)

    except requests.RequestException as e:
        print(f"⚠️ Error descargando {nombre}: {e}")
    except Exception as e:
        print(f"⚠️ Error procesando {nombre}: {e}")
    finally:
        if os.path.exists(tmp_path):
            os.remove(tmp_path)

    return ""

# LISTAR Y PROCESAR ARCHIVOS

In [ ]:
tickers_global = set()

for idx, item in enumerate(data.get("value", [])):
    if idx >= 1 or "folder" in item:  # Solo procesa el primer documento
        continue

    texto = procesar_archivo_temporal(item)
    if texto.strip():
        try:
            sugerencias = sugerir_tickers(texto, max_etiquetas=10)
            tickers_global.update([s.strip() for s in sugerencias if s.strip()])
        except Exception as e:
            print(f"⚠️ Error generando etiquetas para {item.get('name')}: {e}")

print("\n✅ Tickers finales:\n", sorted(tickers_global))